In [2]:
import pandas as pd
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.config import list_physical_devices
from sklearn.utils.class_weight import compute_class_weight

In [7]:
len(list_physical_devices('GPU'))

1

In [2]:
metadata = pd.read_csv('data/Chest_xray_Corona_Metadata.csv').drop('Unnamed: 0', axis=1)

In [3]:
metadata.head()

,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [4]:
combi = list(set(zip(metadata.Label.tolist(), metadata.Label_1_Virus_category.tolist())))
combi

[('Pnemonia', 'bacteria'),
 ('Pnemonia', 'Stress-Smoking'),
 ('Pnemonia', 'Virus'),
 ('Normal', nan)]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8,8))
axs = axs.reshape(-1)
i = 0
for c in combi:
    label, cat = c[0], c[1]
    filename = (
        metadata[
            (metadata['Label'] == label) &
            ((metadata['Label_1_Virus_category'] == cat) | (metadata['Label_1_Virus_category')
        ]
        .isnull()))]
        .iloc[0]
        .X_ray_image_name
    )
    img = Image.open(f'data/train/{filename}')
    img.show()
    axs[i].imshow(img)
    axs[i].title.set_text(f'{c[0]} - {c[1]}')
    i += 1

plt.show()

In [1]:
image_root_path = 'data/'

In [ ]:
# move images

folder_map = {'Normal': 'class_0/', 'Pnemonia': 'class_1/'}
for row in metadata.itertuples():
    filename = row[1]
    folder = folder_map[row[2]]
    train_test = row[3].lower() +'/'
    current_location = f'{image_root_path}{train_test}{filename}'
    new_location = f'{image_root_path}{train_test}{folder}{filename}'
    os.rename(current_location, new_location)

In [ ]:
batch_size = 50
input_shape = (224, 224, 3)
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
def prepare_data(ds, shuffle=False, augment=False):
    ds = ds.map(lambda x, y: (preprocess_input(x), y))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(batch_size)
    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE
        )
    return ds.prefetch(buffer_size=AUTOTUNE)